In [16]:
import pandas as pd
import numpy as np
from pathlib import Path
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [18]:
data = Path('../data/model_data.parquet')
df = pd.read_parquet(data)

print("Data loaded:", df.shape)
df.head()

Data loaded: (4076302, 14)


,Purchase price,Area,price_per_m,suburb_median_price,suburb_price_growth,zoning_median_price,zoning_growth,sale_year,sale_month,sale_quarter,Year,Property post code,Zoning,price_log
100715,283800,442.6,641.211026,280000.0,0.0,362000.0,0.0,2000,11,4,2000,2150,A,12.556029
546864,385000,NaN,<NA>,470000.0,0.0,<NA>,<NA>,2000,3,1,2000,2000,None,12.861001
743963,385000,NaN,<NA>,437500.0,0.0,<NA>,<NA>,2000,3,1,2000,2000,None,12.861001
42136,787500,414.2,1901.255432,407500.0,0.0,440000.0,0.0,2000,12,4,2000,2134,B,13.57662
17423,160000,897.9,178.193563,134975.0,0.0,362000.0,0.0,2000,12,4,2000,2444,A,11.982935


In [ ]:
train = df[df['Year'] < 2020].copy()
valid = df[(df['Year'] >= 2020) & (df['Year'] < 2024)].copy()
test = df[df['Year'] >= 2024].copy()

print("Train:", len(train), "Valid:", len(valid), "Test:", len(test))

Train: 3091890 Valid: 674750 Test: 0
